In [31]:
# import modules
import numpy as np
import astropy.units as u
import astropy.table as tbl
from ReadFile import Read
import CenterofMass


In [41]:
class MassProfile:
# Class to define mass profile of a given galaxy at SnapNumber 0 to determine each 
# falaxy's rotation curve
    
    
    def __init__(self, galaxy, Snap):
    # Initialize the instance of this Class with the following properties:
    # galaxy and Snapshot
    
        # inputs to recontruct the filenumber to the value "000"
        ilbl = '000' + str(Snap)
        # remove all but the last 3 digits
        ilbl = ilbl[-3:]
        self.filename = "%s_"%(galaxy) + ilbl + '.txt'
        
        
        # read data in the given file using Read
        self.time, self.total, self.data = Read(self.filename) 
        
        self.gname = galaxy
        
      
        
        #read data for x,y,z positions and mass with correct units
        
        self.m = self.data['m']
       
        #Positions
        self.x = self.data['x'] * u.kpc
        self.y = self.data['y'] * u.kpc
        self.z = self.data['z'] * u.kpc
        
        # Store the name of the galaxy as a global property
        self.gname = galaxy
        
        # Function that computer the mass enclosed within a given radius of the COM position
        # for a specified galaxy an a specified component of that galaxy
        
    def MassEnclosed(self, ptype, r):
        # Inputs: particle type and radii array in kpc
        # Returns:  mass array in units of Msun
        
        # Reading particle type
        self.index = np.where(self.data['type'] == ptype)
        
        # CenterofMass object 
        COM = Read(CenterofMass(galaxy, ptype))
            
        # Indexing positions with particle type
        xpos= self.x[self.index]
        ypos= self.y[self.index]
        zpos= self.z[self.index]

        # Computing the radius
        radius = np.sqrt(xpos**2 + ypos**2 + zpos**2)
            
        #returns the mass withing the given radius
        mass_rcut = self.mass[rindex] 
            
        return mass_rcut
            
        
        
        
        
        

In [42]:
MWMP = MassProfile("MW", 0)

In [43]:
MW_M_ENC = MWMP.MassEnclosed(1,1 )

TypeError: expected str, bytes or os.PathLike object, not module

In [ ]:
print(MW_M_ENC)